## Data Preparation: 
- https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results
- https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities

### Quick Overlook

In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
df_athletes = pd.read_csv("athlete_events.csv")
df_regions = pd.read_csv("noc_regions.csv")
df_cities = pd.read_html("https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities", header = 0)[1]

- There are missing values in athlete dataset and it also contains redundant data.

In [47]:
df_athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [43]:
df_athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
ID        271116 non-null int64
Name      271116 non-null object
Sex       271116 non-null object
Age       261642 non-null float64
Height    210945 non-null float64
Weight    208241 non-null float64
Team      271116 non-null object
NOC       271116 non-null object
Games     271116 non-null object
Year      271116 non-null int64
Season    271116 non-null object
City      271116 non-null object
Sport     271116 non-null object
Event     271116 non-null object
Medal     39783 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


### Data Cleansing

In [44]:
for col in df_athletes.columns:
    df_athletes[col] = df_athletes[col].astype('category')

In [45]:
df_athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
ID        271116 non-null category
Name      271116 non-null category
Sex       271116 non-null category
Age       261642 non-null category
Height    210945 non-null category
Weight    208241 non-null category
Team      271116 non-null category
NOC       271116 non-null category
Games     271116 non-null category
Year      271116 non-null category
Season    271116 non-null category
City      271116 non-null category
Sport     271116 non-null category
Event     271116 non-null category
Medal     39783 non-null category
dtypes: category(15)
memory usage: 18.6 MB


- After 

### What is the overall ratio of female to male atheletes? 

- The overall ratio is 0.33. After removing the duplicate names of athletes, which means I remove the situation that one athlete has joined many events. I divide the number of male athletes from the female athletes, which represents the overall mutiples of male athletes than female athletes.

In [48]:
athlete_mask = df_athletes.drop_duplicates(keep="first" ,subset="Name")
len(athlete_mask[athlete_mask.Sex == "F"]) / len(athlete_mask[athlete_mask.Sex == "M"])

0.334601250086674

###  Which athelete competed in most number of events? 
In which events did the athelete participate, and for what range of years? Which country did the athlete represent?

In [49]:
df_athletes.Name.value_counts().head()

Robert Tait McKenzie        58
Heikki Ilmari Savolainen    39
Joseph "Josy" Stoffel       38
Ioannis Theofilakis         36
Takashi Ono                 33
Name: Name, dtype: int64

- In df_athlete dataframe, I simply calculated the counts of athlete names.

In [50]:
Most_num_athlete = df_athletes[df_athletes.Name == "Robert Tait McKenzie"]
Most_num_athlete.NOC.head(1)
df_regions[df_regions.NOC == "CAN"].region

36    Canada
Name: region, dtype: object

- To find the country, I searched the whole dataset with the name I have found above.

In [51]:
Most_num_athlete.Event.drop_duplicates()

154798                   Art Competitions Mixed Sculpturing
154799          Art Competitions Mixed Sculpturing, Statues
154806    Art Competitions Mixed Sculpturing, Medals And...
154807    Art Competitions Mixed Sculpturing, Unknown Event
154850           Art Competitions Mixed Sculpturing, Medals
Name: Event, dtype: category
Categories (765, object): [Aeronautics Mixed Aeronautics, Alpine Skiing Men's Combined, Alpine Skiing Men's Downhill, Alpine Skiing Men's Giant Slalom, ..., Wrestling Women's Heavyweight, Freestyle, Wrestling Women's Light-Heavyweight, Freestyle, Wrestling Women's Lightweight, Freestyle, Wrestling Women's Middleweight, Freestyle]

In [52]:
Most_num_athlete.Year.drop_duplicates()

154798    1912
154799    1928
154806    1932
154850    1936
154855    1948
Name: Year, dtype: category
Categories (35, int64): [1896, 1900, 1904, 1906, ..., 2010, 2012, 2014, 2016]

- Athlete who completed most number of games is Robert Tait McKenzie, he is from Canada, and he is active from 1912 - 1928. The types of his events include Medals, Status, and many unknown events.

###  Which sport has the highest median athlete age? 

In [56]:
df_athletes.groupby("Sport").Age.median().sort_values(ascending = False).head()

Sport
Roque               59.0
Art Competitions    45.0
Alpinism            38.0
Polo                35.0
Shooting            33.0
Name: Age, dtype: float64

- I grouped athletes dataset by 'Sport', and calculated the median value of age for those groups. According to the result, sport which has the highest athlete age is Roque.

###  In which events has the Netherlands won the most medals?

In [57]:
Noc_N = df_regions[df_regions.region == 'Netherlands'].NOC.values[0]
medal_mask = df_athletes.dropna(subset = ["Medal"])
medal_mask[medal_mask.NOC == Noc_N].Event.value_counts().head()

Hockey Women's Hockey                              128
Hockey Men's Hockey                                127
Swimming Women's 4 x 100 metres Freestyle Relay     55
Rowing Women's Coxed Eights                         45
Football Men's Football                             42
Name: Event, dtype: int64

- Hockey Women's Hockey, in which Netherlands has won 128 medals. First I chose the data with region = Netherlands, then I drop the empty values, at last, I calcluated the medals of different events.

### Which countries have more female medalists than male medalists?

- Grouped by sex, I chose to use the 'top' module to repersent the result, as the question is only asking which country has more female medalists than male medalists. The 'top' repersents the one which has more counts. Thus, those countries are Burundi, Belarus, Bahrain, Costa Rica, China, Kosovo, Montenegro, Mozambique, Peru, North Korea, Sri Lanka, Ukraine, Zimbabwe

In [59]:
medalist = medal_mask.drop_duplicates(subset = "Name", keep = "first").groupby("NOC")
countries = medalist.Sex.describe()[medalist.Sex.describe().top == "F"].index.values
print(countries)
newlst = []
for noc in countries:
    newlst.append( df_regions[df_regions.NOC == noc].region)
newlst

['BDI' 'BLR' 'BRN' 'CHN' 'CRC' 'HKG' 'KOS' 'MNE' 'MOZ' 'PER' 'PRK' 'SGP'
 'SRI' 'UKR' 'ZIM']


[18    Burundi
 Name: region, dtype: object, 25    Belarus
 Name: region, dtype: object, 30    Bahrain
 Name: region, dtype: object, 41    China
 Name: region, dtype: object, 49    Costa Rica
 Name: region, dtype: object, 88    China
 Name: region, dtype: object, 110    Kosovo
 Name: region, dtype: object, 136    Montenegro
 Name: region, dtype: object, 138    Mozambique
 Name: region, dtype: object, 157    Peru
 Name: region, dtype: object, 164    North Korea
 Name: region, dtype: object, Series([], Name: region, dtype: object), 186    Sri Lanka
 Name: region, dtype: object, 212    Ukraine
 Name: region, dtype: object, 229    Zimbabwe
 Name: region, dtype: object]

### Which country won the most medal points in each of the 2016 Summer Olympics?

In [60]:
score_medal_undropped = medal_mask[(medal_mask.Year == 2016)& (medal_mask.Season == "Summer")]
score_medal = score_medal_undropped.drop_duplicates(subset = ['Event', 'NOC','Medal'], keep = 'first')
score_medal["Score"] = 0
score_medal.loc[score_medal.Medal == "Gold", "Score"] = 3
score_medal.loc[score_medal.Medal == "Bronze", "Score"] = 1
score_medal.loc[score_medal.Medal == "Silver", "Score"] = 2
score_medal.groupby('NOC').Score.sum().sort_values(ascending = False).head()

/Users/chenjieqing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chenjieqing/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


NOC
USA    250
GBR    144
CHN    140
RUS    111
GER     86
Name: Score, dtype: int64

- As the result shown above, the first column represents the country, and the second column repersents the total points that country has got. Thus, USA has won the most points in each of the 2016 Summer Olympics. Adding a new column which represents the overall score of one country, the task becomes to find the exact number of three types of medals of each country. I find that there are many duplicats in the dataframe, like each member of one football team is recorded Individually. So I dropped those duplicate by using drop_duplicate function. And calculate the final points.

### For each year in which games were held, what proportion of gold medals did the host country win?  

As the df_athlete dataframe has already contained the city name of the host country, our job is to find the country name given the city name, then find the NOC of this country. After that, we will have chance to find the total number of gold medals of the host country.

In [66]:
df_cities_mask = df_cities[['City', 'Country']]
df_cities_mask.City = df_cities_mask.City.str.replace('\[.\]','')
df_cities_mask.drop_duplicates(subset = 'City',inplace = True)
df_cities_mask.head(3)

/Users/chenjieqing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country
0,Athens,Greece
1,Paris,France
2,St. Louis,United States


- First, to find the country name given the city name, we will need df_cities dataframe. In this dataframe, I have found that because of the citation from wikipedia, some cities' names have an unexpected citation mark, like "St. Louis[a]", "London[c]". So I decide to use regular expression to find those unexpected marks, and remove them from this dataframe.

In [67]:
temp_df = medal_mask.drop_duplicates(subset = ['Event', 'NOC'], keep = 'first')
df_all_games = temp_df[temp_df.Medal == "Gold"]
add_country_medalist = pd.merge(df_all_games, df_cities_mask, how = "left",  on = 'City' )
add_country_medalist.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country
0,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,France
1,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold,United Kingdom
2,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold,United Kingdom


- Second, we need to find the country name in athlete dataframe using the cities mask dataframe we have cleaned above. So I choose to merge these two dataframe based on City.

In [68]:
na_cites_lst = add_country_medalist[add_country_medalist.Country.isna()].City.unique()
print(na_cites_lst)

def remove_na_cities(City_lst):
    correct_name = {"Moskva":"Moscow","Roma":"Rome", "Torino":"Turin", "Melbourne":"MelbourneStockholm"
                      , "Athina":"Athens",  "Sankt Moritz":"St. Moritz", "Antwerpen": 'Antwerp'}
    for i  in range(len(City_lst)):
        if City_lst[i] in correct_name:
            City_lst[i] = correct_name[City_lst[i]]
    return City_lst

add_country_medalist.City = remove_na_cities(list(add_country_medalist.City))

['Melbourne' 'Moskva' 'Roma' 'Athina' 'Antwerpen' 'Torino' 'Sankt Moritz']


In [69]:
add_country_medalist_cleadned = pd.merge(add_country_medalist, df_cities_mask, how = "left",  on = 'City' )
add_country_medalist_cleadned.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country_x,Country_y
0,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,France,France
1,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold,United Kingdom,United Kingdom
2,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold,United Kingdom,United Kingdom


- Then I found that some cities didn't match with a country, which contained NA in the Country column. So I decided to find which countries those cities belong to and assign them a country name. Here, I implemented a function called remove_na_cities. As you can see in the eighth row, 'Melbourne' has been changed to 'MelbourneStockholm', and also the country name is correctly assigned.

In [70]:
add_country_medalist_cleadned.drop(columns = ['Country_x', 'ID'], inplace = True)

In [72]:
add_country_medalist_cleadned.rename(columns = {'Country_y':'Host_Country', 'City': 'Host_City'}, inplace = True)
add_country_medalist_cleadned.head(3)

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,Host_City,Sport,Event,Medal,Host_Country
0,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,France
1,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold,United Kingdom
2,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold,United Kingdom


- Rename the new column.

In [ ]:
temp1 = add_country_medalist_cleadned[['Team', 'NOC']].drop_duplicates(subset = 'NOC')

In [23]:
new_region = add_NOC_country[['Team', 'NOC_x']]
new_region = new_region.drop_duplicates(subset = 'NOC_x')
# new_region[new_region.NOC_x == '']
df_host_noc = add_NOC_country[['NOC_y', 'Country_y']].drop_duplicates()
df_NOC_code = pd.merge(new_region,df_host_noc , left_on = 'NOC_x', right_on = 'NOC_y',how = 'right')
df_NOC_code

NameError: name 'add_NOC_country' is not defined

In [24]:
na_NOC_lst = add_NOC_country[add_NOC_country['NOC_y'].isna()].Country_y.unique()
na_NOC_lst

correct_name = {
    'United Kingdom': 'GBR',
    'Australia\xa0Sweden' : 'AUS+SEN',
    'United States' :  'USA',
    'Soviet Union' : 'URS',
    'West Germany' : 'FRG',
    'Yugoslavia': 'YUG',
    'Nazi Germany' : 'GER'
}
Country_lst = add_NOC_country.Country_y
NOC_lst = add_NOC_country.NOC_y
for i  in range(len(Country_lst)):
    if Country_lst[i] in correct_name:
        NOC_lst[i] = correct_name[Country_lst[i]]
add_NOC_country.NOC_y = NOC_lst

NameError: name 'add_NOC_country' is not defined

In [25]:
add_NOC_country.head()

NameError: name 'add_NOC_country' is not defined

- Third, I need to find the NOC of those countries. But when I dig into the dataframe, I find that there are also many NA in NOC column. The reason is that in region dataframe, for example, 'United Kingdom' is called 'UK', and 'United States' is called 'USA'. To solve that problem, I decide to clean the dataframe by using columns 'Team' and 'NOC_x' as a new reference of NOC.

In [253]:
df_regions[df_regions.region == 'Nazi Germany']

,NOC,region,notes


(Use this space to explain your answers)
- As the result shown above, the first column represents the year, and the second column represents the proportion of gold medals the host country has won. I merged the athlete dataset and the cities dataset by city name and year. After dropping na values, only the host country will be left. Then I calculated the total gold medal of host country and the total medal of host country. and then I calculate the proportion.